In [50]:
import pandas as pd
import psycopg2 
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 10000

run  `ssh -L 5433:localhost:5432 cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city -N` 
to establish ssh tunnel


or to forward socket:
`mkdir /tmp/sql-socket`
`ssh -L /tmp/sql-socket/.s.PGSQL.5432:/run/postgresql/.s.PGSQL.5432 -N cardano-cli@node-1.mainnet.ctl-runtime.staging.mlabs.city`
`psql -h /tmp/sql-socket -U cexplorer -d cdbsync`


In [48]:
conn = psycopg2.connect(
    dbname='cdbsync',
    user='cexplorer',
    host='/tmp/sql-socket',
    port=5432
)

# conn.close()


In [3]:
all_tables_and_columns = """
SELECT
    t.tablename AS table_name,
    a.attname AS column_name
FROM
    pg_tables t
JOIN
    pg_attribute a ON t.tablename = a.attrelid::regclass::text
WHERE
    t.tableowner = 'cexplorer'
    AND a.attnum > 0
    AND NOT a.attisdropped
ORDER BY
    t.tablename,
    a.attnum;
"""
all_tables = pd.read_sql(all_tables_and_columns,conn)

/tmp/nix-shell.wU4exl/ipykernel_56345/3796777492.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_tables = pd.read_sql(all_tables_and_columns,conn)


In [4]:
all_tables

,table_name,column_name
0,ada_pots,id
1,ada_pots,slot_no
2,ada_pots,epoch_no
3,ada_pots,treasury
4,ada_pots,reserves
5,ada_pots,rewards
6,ada_pots,utxo
7,ada_pots,deposits
8,ada_pots,fees
9,ada_pots,block_id


In [18]:
max_time = """
SELECT max(time)
    
FROM
    block
;
"""
pd.read_sql(max_time,conn)

/tmp/nix-shell.wU4exl/ipykernel_56345/478863478.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(max_time,conn)


,max
0,2022-09-22 21:43:26


In [3]:
blocks = """
SELECT
    *
FROM
    block
WHERE
  DATE(time)='2022-04-25'
LIMIT
    3;
"""
pd.read_sql(blocks,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/2541202552.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(blocks,conn)


,id,hash,epoch_no,slot_no,epoch_slot_no,block_no,previous_id,slot_leader_id,size,time,tx_count,proto_major,proto_minor,vrf_key,op_cert,op_cert_counter
0,7165446,"[b'\x0e', b'\xfa', b'y', b'q', b',', b'\x15', ...",334,59278513,353713,7165269,7165445,4981904,76849,2022-04-25 00:00:04,36,6,0,vrf_vk1rt4whnfaazzgxepp5vw0t59msuy8nyg9z6cadfd...,"[b'G', b'\xe2', b'>', b'\xe6', b'\x1b', b'\xfd...",7
1,7165447,"[b'\x94', b'\xc5', b'L', b'\x9c', b'\x84', b'\...",334,59278554,353754,7165270,7165446,7005759,4,2022-04-25 00:00:45,0,6,0,vrf_vk1t0syd45jwwwrfw6jpmc9668nqxmm4lq4wmjcfyq...,"[b'\xf5', b'\x8f', b'\xa6', b'\x1f', b'\x84', ...",0
2,7165448,"[b'\xea', b'.', b'\xf6', b'\xa3', b'I', b'\x8c...",334,59278567,353767,7165271,7165447,6096969,77679,2022-04-25 00:00:58,27,6,0,vrf_vk1v7n8q565aej4vqu3wkx5kcuxs3vslcr08pj70sa...,"[b'\x92', b'6', b'\xbb', b'\x96', b'\x9c', b'\...",6


In [36]:

blocks = """
SELECT
    ENCODE(tx.hash, 'hex') AS hash,
    block.block_no,
    block.time,
    block.tx_count,
    tx.script_size,
    tx.size
    
FROM block
LEFT JOIN tx ON block.id = tx.block_id    
WHERE
  DATE(block.time)='2022-04-25'

"""
df = pd.read_sql(blocks,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/4000378025.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(blocks,conn)


In [38]:
df.sort_values(by = 'size').head(2)


,hash,block_no,time,tx_count,script_size,size
77027,1ee30f055fc21ac067162b261bdb5034425bafb6e0b2e642554d0409c677c147,7168520,2022-04-25 19:30:44,32,0.0,190.0
44329,2075842a95a55d95d2bcb039dc1ad856f8fbe7cb73c68e60de1f5115b35310f7,7167412,2022-04-25 12:35:55,30,0.0,190.0


In [40]:
df.sort_values(by = 'script_size').head(2)

,hash,block_no,time,tx_count,script_size,size
0,f5d967e75124fcbb288e163a68f9da0c964afca2ff89ee9d5ffe38466afd9b92,7165269,2022-04-25 00:00:04,36,0.0,258.0
51491,86bac65004f7393808642027123cc0a3f74ed39b47f034e0a6b2b401dbf991fa,7167688,2022-04-25 14:19:57,31,0.0,876.0


In [33]:
df.to_pickle('df.pkl')

In [44]:

temp = """
SELECT
    ENCODE(tx.hash, 'hex') AS hash,
    tx.script_size,
    tx.size
    
    
FROM tx
WHERE script_size > 12000
LIMIT 10;
"""
pd.read_sql(temp,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/3580242147.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(temp,conn)


,hash,script_size,size
0,cb3828b8a1ecdfc269184548febddefa41fd582754b6b66d881865ba17288fdf,12997,14508
1,e3df8b42afccb3aacc83da65e0fb9b1907f095d50c8af940bea67cd67eb2dba0,13223,14459
2,347eb42d2546d56388b5d7390c6bb94da0117a488e8e70096751d68d89743506,13223,14459
3,70e805cbde9910a9a6f1d4f22326c31e6bba4452fb14d852c38f20236ff01f63,13223,14459
4,1ef46843cbdda3271893ab237e33016a9aa63c6eb9f0cd25e0495a221445ef66,13223,14459
5,224308825052ecf065ed5793461b4d55ba7e23eee2d2d5a8d916e96447707c87,13223,14711
6,fda591afa46f8d659a1d9e1691a1df0ed42f166898dc067c1bcb5e6a2331676d,13223,14711
7,0b3ebe7212f459790f9944dc3aaca3a3c1b3dd90d6027434f846c354cfd6f971,13223,14711
8,4992412407e7fbf7cbd2af87482ea718f4cab2f6318357351d13de5b89c54f4e,13223,14711
9,8f0695e1ed1b2d3a6a193a153be8b1badfb9e0f0e71e5648d1a3c809e40d0376,13223,14711


In [51]:

temp = """
SELECT
  ENCODE(hash, 'hex') AS hash,  *
    
FROM script
LIMIT 10;
"""
pd.read_sql(temp,conn)

/tmp/nix-shell.wU4exl/ipykernel_56943/836493797.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(temp,conn)


,hash,id,tx_id,hash,type,json,bytes,serialised_size
0,65c197d565e88a20885e535f93755682444d3c02fd44dd70883fe89e,1,3291942,"[b'e', b'\xc1', b'\x97', b'\xd5', b'e', b'\xe8', b'\x8a', b' ', b'\x88', b'^', b'S', b'_', b'\x93', b'u', b'V', b'\x82', b'D', b'M', b'<', b'\x02', b'\xfd', b'D', b'\xdd', b'p', b'\x88', b'?', b'\xe8', b'\x9e']",timelock,"{'type': 'all', 'scripts': [{'type': 'sig', 'keyHash': 'a96da581c39549aeda81f539ac3940ac0cb53657e774ca7e68f15ed9'}, {'type': 'sig', 'keyHash': 'ccfcb3fed004562be1354c837a4a4b9f4b1c2b6705229efeedd12d4d'}, {'type': 'sig', 'keyHash': '74fcd61aecebe36aa6b6cd4314027282fa4b41c3ce8af17d9b77d0d1'}]}",None,None
1,00000002df633853f6a47465c9496721d2d5b1291b8398016c0e87ae,2,4666648,"[b'\x00', b'\x00', b'\x00', b'\x02', b'\xdf', b'c', b'8', b'S', b'\xf6', b'\xa4', b't', b'e', b'\xc9', b'I', b'g', b'!', b'\xd2', b'\xd5', b'\xb1', b')', b'\x1b', b'\x83', b'\x98', b'\x01', b'l', b'\x0e', b'\x87', b'\xae']",timelock,"{'type': 'all', 'scripts': [{'slot': 24285375, 'type': 'before'}, {'type': 'sig', 'keyHash': 'e97316c52c85eab276fd40feacf78bc5eff74e225e744567140070c3'}]}",None,None
2,3a9241cd79895e3a8d65261b40077d4437ce71e9d7c8c6c00e3f658e,3,4666649,"[b':', b'\x92', b'A', b'\xcd', b'y', b'\x89', b'^', b':', b'\x8d', b'e', b'&', b'\x1b', b'@', b'\x07', b'}', b'D', b'7', b'\xce', b'q', b'\xe9', b'\xd7', b'\xc8', b'\xc6', b'\xc0', b'\x0e', b'?', b'e', b'\x8e']",timelock,"{'type': 'all', 'scripts': [{'type': 'sig', 'keyHash': '26bacc7b88e2b40701387c521cd0c50d5c0cfa4c6c6d7f0901395757'}, {'slot': 23069343, 'type': 'before'}]}",None,None
3,02f68378e37af4545d027d0a9fa5581ac682897a3fc1f6d8f936ed2b,4,4666653,"[b'\x02', b'\xf6', b'\x83', b'x', b'\xe3', b'z', b'\xf4', b'T', b']', b'\x02', b'}', b'\n', b'\x9f', b'\xa5', b'X', b'\x1a', b'\xc6', b'\x82', b'\x89', b'z', b'?', b'\xc1', b'\xf6', b'\xd8', b'\xf9', b'6', b'\xed', b'+']",timelock,"{'type': 'sig', 'keyHash': '89d555c7a028bc560ea44e52a81c866088566f3a99c9989a5a183940'}",None,None
4,e8e62d329e73190190c3e323fb5c9fb98ee55f0676332ba949f29d72,5,4666655,"[b'\xe8', b'\xe6', b'-', b'2', b'\x9e', b's', b'\x19', b'\x01', b'\x90', b'\xc3', b'\xe3', b'#', b'\xfb', b'\\', b'\x9f', b'\xb9', b'\x8e', b'\xe5', b'_', b'\x06', b'v', b'3', b'+', b'\xa9', b'I', b'\xf2', b'\x9d', b'r']",timelock,"{'type': 'sig', 'keyHash': '8a2da46df46082c69076335093c8062f9f1413f452ec43357f4f2f74'}",None,None
5,ac3f4224723e2ed9d166478662f6e48bae9ddf0fc5ee58f54f6c3229,6,4666656,"[b'\xac', b'?', b'B', b'$', b'r', b'>', b'.', b'\xd9', b'\xd1', b'f', b'G', b'\x86', b'b', b'\xf6', b'\xe4', b'\x8b', b'\xae', b'\x9d', b'\xdf', b'\x0f', b'\xc5', b'\xee', b'X', b'\xf5', b'O', b'l', b'2', b')']",timelock,"{'type': 'all', 'scripts': [{'slot': 23069400, 'type': 'before'}, {'type': 'sig', 'keyHash': '24a24be012b30b349f231a99280537f6216dd39568d6f9490bbc68af'}]}",None,None
6,12e65fa3585d80cba39dcf4f59363bb68b77f9d3c0784734427b1517,7,4666661,"[b'\x12', b'\xe6', b'_', b'\xa3', b'X', b']', b'\x80', b'\xcb', b'\xa3', b'\x9d', b'\xcf', b'O', b'Y', b'6', b';', b'\xb6', b'\x8b', b'w', b'\xf9', b'\xd3', b'\xc0', b'x', b'G', b'4', b'B', b'{', b'\x15', b'\x17']",timelock,"{'type': 'all', 'scripts': [{'slot': 23075571, 'type': 'before'}, {'type': 'sig', 'keyHash': '76cb8de7db67a0cf13ba419e7b571d479a92211ef1a51e3174f85b55'}]}",None,None
7,e12ab5cf12f95cd57b739282d06af9dd61e1b1dde1e06f0c31f02511,8,4666662,"[b'\xe1', b'*', b'\xb5', b'\xcf', b'\x12', b'\xf9', b'\\', b'\xd5', b'{', b's', b'\x92', b'\x82', b'\xd0', b'j', b'\xf9', b'\xdd', b'a', b'\xe1', b'\xb1', b'\xdd', b'\xe1', b'\xe0', b'o', b'\x0c', b'1', b'\xf0', b'%', b'\x11']",timelock,"{'type': 'sig', 'keyHash': '7280bfc12cc21c024a3e8155d3d77227d7bf8f66ab2fa35d5799c163'}",None,None
8,da8c30857834c6ae7203935b89278c532b3995245295456f993e1d24,9,4666667,"[b'\xda', b'\x8c', b'0', b'\x85', b'x', b'4', b'\xc6', b'\xae', b'r', b'\x03', b'\x93', b'[', b'\x89', b""'"", b'\x8c', b'S', b'+', b'9', b'\x95', b'$', b'R', b'\x95', b'E', b'o', b'\x99', b'>', b'\x1d', b'$']",timelock,"{'type':